### Import Libraries

In [11]:
!pip install arxiv
! pip install -U langchain
! pip install -qU langchain[groq]
! pip install langchain_community
! pip install -qU langchain-mistralai
! pip install pypdf
! pip install chromadb
! pip install faiss-cpu
! pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=f51bcd550a2c2d5f8a7a9a00d7037f35608b60f27e838012af5d5c325db1bb52
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


In [ ]:
import os

os.environ['LANGSMITH_TRACING'] = "true"   # for tracing api calls
os.environ['LANGSMITH_ENDPOINT'] = "https://api.smith.langchain.com"  # provide the url where it will trace
os.environ['LANGSMITH_API_KEY'] = "YOUR_LANGCHAIN_API_KEY" # api key to access langchain
os.environ['LANGSMITH_PROJECT'] ="Agentic RAG" # project name on langsmith
os.environ['GROQ_API_KEY'] = "YOUR_GROQ_API_KEY" # LLM api key for
os.environ['MISTRAL_API_KEY'] = "YOUR_MISTRAL_API_KEY"
os.environ['HF_TOKEN'] = 'YOUR_HF_API_KEY'

In [13]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun

### Create Wikipedia Tool

In [14]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [15]:
wiki.name

'wikipedia'

In [16]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_mistralai import MistralAIEmbeddings
from langchain_community.vectorstores import FAISS

In [22]:
!pip install wikipedia

### Create Websearch Tool

In [131]:
loader=WebBaseLoader("https://python.langchain.com")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectorDB = FAISS.from_documents(documents, MistralAIEmbeddings())
retriever = vectorDB.as_retriever()
retriever

/usr/local/lib/python3.11/dist-packages/langchain_mistralai/embeddings.py:181: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


VectorStoreRetriever(tags=['FAISS', 'MistralAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7ad5aab77b90>, search_kwargs={})

In [132]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever, 'langchain_search', "For any questions about langchain you must use this tool")

In [133]:
retriever_tool.name

'langchain_search'

### Create Arixv Tool

In [134]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=500)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

### Combine three tools for Agent

In [135]:
tools = [wiki, arxiv, retriever_tool]

In [136]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/usr/local/lib/python3.11/dist-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=500)),
 Tool(name='langchain_search', description='For any questions about langchain you must use this tool', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x7ad5b587dda0>, retriever=VectorStoreRetriever(tags=['FAISS', 'MistralAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7ad5aab77b90>, search_

### Load LLama3.1 Model

In [137]:
from langchain.chat_models import init_chat_model

llm = init_chat_model('llama-3.1-8b-instant', model_provider='groq')
llm


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7ad5aaba2bd0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7ad5aab61110>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [138]:
from langchain import hub

prompt = hub.pull('hwchase17/openai-functions-agent')
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

### **create Agent**

In [139]:
from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)

In [140]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

### **Create Agent Exector**

In [141]:
from langchain.agents import AgentExecutor

agent_excecutor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_excecutor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

### Excecute Query related to Web

In [109]:
agent_excecutor.invoke({'input': 'Who was Benjamin Franklin?'})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Benjamin Franklin'}`


Page: Benjamin Franklin
Summary: Benjamin Franklin (January 17, 1706 [O.S. January 6, 1705] – April 17, 1790) was an American polymath: a writer, scientist, inventor, statesman, diplomat, printer, pub
Invoking: `wikipedia` with `{'query': 'Benjamin Franklin biography'}`


Page: The Autobiography of Benjamin Franklin
Summary: The Autobiography of Benjamin Franklin is the traditional name for the unfinished record of his own life written by Benjamin Franklin from 1771 to
Invoking: `wikipedia` with `{'query': 'Benjamin Franklin achievements'}`


Page: Franklin Institute Awards
Summary: The Franklin Institute Awards (or Benjamin Franklin Medal) is an American science and engineering award presented by the Franklin Institute, a science museum i
Invoking: `wikipedia` with `{'query': 'Benjamin Franklin legacy'}`


Page: Benjamin F. McAdoo
Summary: Benjamin Franklin McAdoo Jr. (October 29, 19

{'input': 'Who was Benjamin Franklin?',
 'output': 'An American polymath, Benjamin Franklin was a writer, scientist, inventor, statesman, diplomat, printer, publisher, and musician.'}

### Excecute Query related to Research paper

In [111]:
agent_excecutor.invoke({"input":"which paper is this 1706.03762 ?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks in an encoder-decoder configuration. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer, base
Invoking: `arxiv` with `{'query': 'Transformer model'}`


Published: 2012-10-23
Title: Model Validation in Ontology Based Transformations
Authors: Jesús M. Almendros-Jiménez, Luis Iribarne
Summary: Model Driven Engineering (MDE) is an emerging approach of software
engineering. MDE emphasizes the construction of models from which the
implementation should be derived by applying model transformations. The


{'input': 'which paper is this 1706.03762 ?',
 'output': "It seems that the paper '1706.03762' is a seminal work on the Transformer model, which was first introduced in 2017. The authors of this paper, Ashish Vaswani et al., proposed a new simple network architecture that is based on self-attention mechanisms, which has since become a cornerstone of many natural language processing and machine learning applications."}

### Excecute Query related to Langchain 

In [146]:
response = agent_excecutor.invoke({"input":"what are the different Chat models in langchain ?"})
response



> Entering new AgentExecutor chain...

Invoking: `langchain_search` with `{'query': 'Chat models in Langchain'}`


outputs to chat modelsHow to pass run time values to toolsHow to stream events from a toolHow to stream tool callsHow to convert tools to OpenAI FunctionsHow to handle tool errorsHow to use few-shot prompting with tool callingHow to add a human-in-the-loop for toolsHow to bind model-specific toolsHow to trim messagesHow to create and query vector storesConceptual guideAgentsArchitectureAsync programming with langchainCallbacksChat historyChat modelsDocument loadersEmbedding modelsEvaluationExample selectorsFew-shot promptingConceptual guideKey-value storesLangChain Expression Language (LCEL)MessagesMultimodalityOutput parsersPrompt TemplatesRetrieval augmented generation (RAG)RetrievalRetrieversRunnable interfaceStreamingStructured outputsTestingString-in, string-out llmsText splittersTokensTool callingToolsTracingVector storesWhy LangChain?Ecosystemü¶úüõ†Ô∏è LangSmith

{'input': 'what are the different Chat models in langchain ?',
 'output': 'Based on the provided information, the different Chat models in Langchain are:\n\n1. Groq\n2. OpenAI\n3. Anthropic\n4. Azure\n5. Google Vertex AI\n6. AWS\n7. Cohere\n8. NVIDIA\n9. Fireworks AI\n10. Mistral AI\n11. Together AI\n12. IBM Watson\n13. xDabra\n14. Databricks\n\nPlease note that this list may not be exhaustive, and Langchain might support other chat models as well.'}

### Print Agent Response

In [147]:
print(response['output'])

Based on the provided information, the different Chat models in Langchain are:

1. Groq
2. OpenAI
3. Anthropic
4. Azure
5. Google Vertex AI
6. AWS
7. Cohere
8. NVIDIA
9. Fireworks AI
10. Mistral AI
11. Together AI
12. IBM Watson
13. xDabra
14. Databricks

Please note that this list may not be exhaustive, and Langchain might support other chat models as well.
